# Data Processing with DataFrame 

[DataFrames](https://docs.microsoft.com/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview) are an easy way to explore and manipulate data. We can load, convert, and combine data.  

Let's take a look at the following examples of hockey data. We have information on US and European players, plus a list of player salaries. Let's combine all of this into a single dataset for predicting salaries. Let's normalize the data of US and European players, combine them into one source, and then union with player salaries.  

## Load packages

Get data frame api, visualization and formatting.

In [ ]:
// using nightly-build
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json"
#r "nuget: Microsoft.ML, 3.0.0-preview.23504.1"
#r "nuget: Microsoft.Data.Analysis, 0.21.0-preview.23504.1"

## Inspect data

Let's get started loading our data on the players from the US into a DataFrame.

#### Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [ ]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [ ]:
using System.Linq;
using Microsoft.Data.Analysis;

var usaDataPath = EnsureDataSetDownloaded("usa_hockey.csv"); 
var usaDf = DataFrame.LoadCsv(usaDataPath);

DataFrames store data as a collection of columns. This makes it easy to interact with the data. To get a preview of the columns, we'll run `Info()`.

Here we have a information about the players, where they were born, their height and weight, etc. We also have statistics on their play, like the number of goals/assists, points scored, and games played. 

We can already see that some fields have fewer values than other. Draft year is only available for 7 out of 10 players. 

In [ ]:
usaDf.Info()

Take a look at the actual data. Everything looks pretty normal, but there are some null values. 

In [ ]:
usaDf

### Europe data

We also have data for some European players. Let's preview the data and see how it's different. 

In [ ]:
var europeDataPath = EnsureDataSetDownloaded("europe_hockey.csv"); 
var europeDf = DataFrame.LoadCsv(europeDataPath);
europeDf.Info()

The values look very similar to the US players. One difference is height is in centimeters and weight is in kilograms. Some of the values in this file are missing too. 

In [ ]:
europeDf

Looking at the acutal data we can see that birthday is in a different format YY-MM-DD format. 

Let's dive in a little further and see which countries are represented here. 

In [ ]:
europeDf["Nat"].ValueCounts()

## Convert data

Now that we know something about the data let's normalize and combine these data sources into one. 

Europe players have height and weight in kilograms and centimeters. Since these are players are in the NHL, let's convert these values into pounds and inches to match the American players.

In [ ]:
europeDf["Weight_kg"] = europeDf["Weight_kg"].Multiply(2.20462262).Round(); 
europeDf["Height_cm"] = europeDf["Height_cm"].Divide(2.54).Round();

europeDf["Weight_kg"].SetName("Weight"); 
europeDf["Height_cm"].SetName("Height");

Let's also convert the European birthday to the American format for consistency. 

In [ ]:
// There are birthdays in the format "1999/1/1" and "1999-1-1". Include both splitter characters. 
char[] delimiterChars = { '-', '/'};

var birthday = 
    ((IEnumerable<string>)europeDf["Birthday"])
        .Select(x => x.Split(delimiterChars));

europeDf["Birthday"] = new StringDataFrameColumn("Birthday", birthday.Select(x => x[0] + "-" + x[2] + "-" + x[1]));

To combine the DataFrames they need to look identical. Let's sample some data and confirm the shape is the same. 

In [ ]:
europeDf.Sample(1)

In [ ]:
usaDf.Sample(1)

## Combine data sources

In [ ]:
var allPlayersDf = usaDf.Clone();
europeDf.Rows.ToList<DataFrameRow>().ForEach(row => {allPlayersDf.Append(row, /*append in place*/ true);})

We now have one DataFrame with all the player information. 

In [ ]:
allPlayersDf

The lists are combined together, but we have some missing data for Draft Year and Overall Draft position. Let's replace those nulls so they don't mess up our training. 

In [ ]:
// We're missing information in DraftYear and OverallDraft
allPlayersDf["DraftYear"] = allPlayersDf["DraftYear"].FillNulls(2000); 

var medianDraft = allPlayersDf["OverallDraft"].Median();
allPlayersDf["OverallDraft"] = allPlayersDf["OverallDraft"].FillNulls(medianDraft); 

allPlayersDf

Now that our player information looks good, let's take a look at the salary data we're trying to union with.

This file is in JSON format. DataFrames don't support loading from JSON yet. There is a work around. It's fairly easy to convert from IDataView to DataFrame. Let's use that method to get what we need. 

In [ ]:
public class PlayerSalary
{
    public string Name { get; set; }

    public float Salary { get; set; }
}

In [ ]:
using System.Text.Json;
using System.IO; 
using Microsoft.ML; 

// Read in JSON file
string jsonString = File.ReadAllText(@"data/playerSalary.json");
var players = JsonSerializer.Deserialize<List<PlayerSalary>>(jsonString);

// Load it into an IDataView
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromEnumerable<PlayerSalary>(players);

// Convert to a DataFrame
var playerSalaryDf = data.ToDataFrame();

playerSalaryDf

### Convert two columns into one

Our player salary information has the name format "First Last". The player details has two separate columns for first and last name. Let's fix this. 

In [ ]:
// Grab first and last name. Zip them together into one list. 
var firstNames = (IEnumerable<string>)allPlayersDf["First Name"]; 
var lastNames = (IEnumerable<string>)allPlayersDf["Last Name"];
var fullNames = firstNames.Zip(lastNames, (first, last) => first + " " + last); 

// Create the new column from the combined names
allPlayersDf["FullName"] = new StringDataFrameColumn("FullName", fullNames); 

// Cleanup the unneeded first and last name columns 
allPlayersDf.Columns.Remove("First Name"); 
allPlayersDf.Columns.Remove("Last Name"); 

### Merge two DataFrames together
 
Join the player salaries with player information. We will join on the Name information. 

In [ ]:
// Merge the player salary information with the player details. 
var allPlayersWithSalaries = allPlayersDf.Merge(playerSalaryDf, new string[] {"FullName"}, new string[] {"Name"});

// We now have two columns with names. Remove one. 
allPlayersWithSalaries.Columns.Remove("Name");

// Just for fun, let's put them in alphabetical order 
allPlayersWithSalaries = allPlayersWithSalaries.OrderBy("FullName"); 

In [ ]:
allPlayersWithSalaries

Everything looks really good, expect we have on player without a salary. That won't be useful to us. Let's drop this null value from the table. 

In [ ]:
allPlayersWithSalaries = allPlayersWithSalaries.DropNulls();
allPlayersWithSalaries

## Save results

We succesfully combined three different sources into one. Now it's time to save our results into a .csv file that can be used for training. 

In [ ]:
DataFrame.WriteCsv(allPlayersWithSalaries, "allPlayers.csv", ',')